In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import numpy as np
from scipy import stats

import probability_distributions
import maximum_nudges
import evolutionary_algorithms as ea
import maximum_nudges_evolutionary as ev_max_nudges

In [17]:
#distribution parameters
input_variables = 4
number_of_states = 5
nudge_size = 0.01

#generate both input and conditional output with Dirichlet weights
distribution_shape = [number_of_states]*input_variables
total_number_of_states = reduce(lambda x,y: x*y, distribution_shape)
input_dist = np.random.dirichlet([1]*total_number_of_states)
input_dist = np.reshape(input_dist, distribution_shape)
cond_shape = [number_of_states]*(input_variables+1)
cond_output = [
    probability_distributions.compute_joint_uniform_random((number_of_states,))
    for i in range(number_of_states**(input_variables))
]
cond_output = np.array(cond_output)
cond_output = np.reshape(cond_output, cond_shape)


In [18]:
#synegistic nudge optimization
number_of_generations = 100 
population_size = 10
number_of_children = 20 
generational = True 
mutation_size = nudge_size/4
parent_selection_mode = "rank_exponential"
#parent_selection_mode = None
start_mutation_size = nudge_size/25
change_mutation_size = start_mutation_size/10
mutations_per_update_step = 10


In [19]:
#distribution parameters
input_variables = 4
number_of_states = 5
nudge_size = 0.01

#generate both input and conditional output with Dirichlet weights
distribution_shape = [number_of_states]*input_variables
total_number_of_states = reduce(lambda x,y: x*y, distribution_shape)
input_dist = np.random.dirichlet([1]*total_number_of_states)
input_dist = np.reshape(input_dist, distribution_shape)
cond_shape = [number_of_states]*(input_variables+1)
cond_output = [
    probability_distributions.compute_joint_uniform_random((number_of_states,))
    for i in range(number_of_states**(input_variables))
]
cond_output = np.array(cond_output)
cond_output = np.reshape(cond_output, cond_shape)

synergetic_nudge = ev_max_nudges.SynergisticNudge(input_dist, np.copy(input_dist), cond_output,
                 nudge_size, mutations_per_update_step, start_mutation_size,
                 change_mutation_size, timestamp=0)

In [20]:
old_input = np.copy(input_dist)

#create the initial population
synergistic_nudges = []
for _ in range(population_size):
    new_synergistic_nudge = ev_max_nudges.SynergisticNudge.create_nudge(
        input_dist, cond_output, nudge_size, mutations_per_update_step, 
        start_mutation_size, change_mutation_size, timestamp=0
    )
    synergistic_nudges.append(new_synergistic_nudge)

for synergistic_nudge in synergistic_nudges:
    synergistic_nudge.evaluate()

print("initial impact synergistic nudge {}".format(
    ea.sort_individuals(synergistic_nudges)[0].score
))

#start the optimization process
find_max_synergistic_nudge = ev_max_nudges.FindMaximumSynergisticNudge(
    generational, number_of_children, parent_selection_mode
)
max_synergistic_nudge = find_max_synergistic_nudge.get_max_nudge(
    synergistic_nudges, number_of_generations
)
print("the found max impact for a synergistic nudge {}".format(
    max_synergistic_nudge.score
))


initial impact synergistic nudge -0.00104209665191
the found max impact for a synergistic nudge -0.00226557629792


In [21]:
NUDGE_SIZE = 0.01
new_distribution = max_synergistic_nudge.new_distribution
l1_norm_to_old_distribution = np.sum(np.absolute(old_input-max_synergistic_nudge.new_distribution))
print(l1_norm_to_old_distribution)
actual_new_distribution = old_input + (new_distribution-old_input)*(NUDGE_SIZE/l1_norm_to_old_distribution)

0.01


In [22]:
import probability_distributions
for i in range(len(new_distribution.shape)):
    new_marginal = probability_distributions.ProbabilityArray(actual_new_distribution).marginalize(set([i]))
    old_marginal = probability_distributions.ProbabilityArray(old_input).marginalize(set([i]))
    print(np.sum(np.absolute(new_marginal-old_marginal)))

1.38777878078e-16
1.66533453694e-16
1.38777878078e-16
1.11022302463e-16


In [23]:
import nudge_non_causal
nudge_non_causal.find_nudge_impact(old_input, actual_new_distribution, cond_output, measure="absolute")

0.0022655762979190697